In [ ]:
# config
file_path = 'data.csv'
columns = [
    'name',
    'student_id',
    '周一上午',
    '周一下午',
    '周二上午',
    '周二下午',
    '周三上午',
    '周三下午',
    '周四上午',
    '周四下午',
    '周五上午',
    '周五下午'
]

week_cnt = 8
each_week_col = [
    [5, 6, 7, 8, 9],
    [10, 11, 12, 13, 14],
    [15, 16, 17, 18, 19],
    [20, 21, 22, 23, 24],
    [25, 26, 27, 28, 29],
    [30, 31, 32, 33, 34],
    [35, 36, 37, 38, 39],
    [40, 41, 42, 43, 44],
]
timeStrDef = [
    '9:00~12:00',
    '14:00~17:00'
]
meta_data_col = [
    2, 3
]
class_name_list = columns[2:]

class_max_cap = 6
student_week_capacity = 1


In [ ]:
import pandas as pd

source_df = pd.read_csv(file_path)
source_df.sort_values(by=[source_df.columns[0]], ascending=True, inplace=True)
source_df.drop_duplicates(subset=['學號'], keep='last', inplace=True, ignore_index=False)
print(source_df.head()) # This will show the first 5 rows of your data

In [ ]:
import pandas as pd

df_array = [pd.DataFrame(columns=columns) for _ in range(week_cnt)]

print(f"陣列的長度為：{len(df_array)}")
print("\n第一個 DataFrame：")
print(df_array[0])


In [ ]:
week_idx = 0
for cols in each_week_col:
    target_col = meta_data_col + cols
    selected_df = source_df.iloc[:, target_col]

    for index, row in selected_df.iterrows():
            tmp_data =  []
            for i in range(len(meta_data_col)):
                tmp_data.append(row.iloc[i])
            for r in row[len(meta_data_col):]:
                for i in timeStrDef:
                    if i in str(r):
                        tmp_data.append(True)
                    else:
                        tmp_data.append(False)
            tmp_df = pd.DataFrame([
                tmp_data,
            ], columns=columns)
            df_array[week_idx] = pd.concat([df_array[0], tmp_df], ignore_index=True)
    week_idx = week_idx + 1

print(df_array[0])

In [ ]:
import networkx as nx

w_ret = {}
w_idx = 1
for w_arr in df_array:
    # 建立一個有向圖
    G = nx.DiGraph()

    source = "students"
    sink = "class"
    G.add_node(source)
    G.add_node(sink)

    for col_name in class_name_list:
        class_node_name = col_name
        G.add_node(class_node_name)
        G.add_edge(class_node_name, sink, capacity=class_max_cap)

    for index, row in w_arr.iterrows():
        student_node_name = row['name']+'_'+str(row['student_id'])
        G.add_node(student_node_name)
        G.add_edge(source, student_node_name, capacity=student_week_capacity)
        for classes_name in class_name_list:
            if row[classes_name]:
                G.add_edge(student_node_name, classes_name, capacity=1)


    flow_value, flow_dict = nx.maximum_flow(G, source, sink)

    # print(f"最大流量值：{flow_value}")

    # 印出每條邊的流量分配
    # print("各邊的流量：")
    # tt = 1
    is_all_student_have_class = True
    empty_student_list = []
    student_list = []
    for u, v_dict in flow_dict.items():
        for v, flow in v_dict.items():
            if u == source and flow == 0:
                # print(f" {tt}  從 {u} 到 {v} 的流量為 {flow}")
                # tt = tt+1
                is_all_student_have_class = False
                empty_student_list.append(v)
            if u != source and v != sink and flow == 1:
                student_list.append((u, v))

    if is_all_student_have_class:
        print(f"week {w_idx} 成功分配")
        # print(len(student_list), student_list)
        w_ret[w_idx] = student_list
    else:
        print(f"week {w_idx} 成功失敗")
        print(empty_student_list)
    w_idx = w_idx + 1

# print(w_ret[1])

In [ ]:
import pandas as pd

for w in range(week_cnt+1):
    # 您的原始資料
    if w == 0:
        continue
    data = w_ret[w]

    # 將資料轉換為 DataFrame，並指定欄位名稱
    df = pd.DataFrame(data, columns=['姓名_學號', '時段'])

    # 將 DataFrame 匯出為 CSV 檔案
    # index=False 避免將索引寫入 CSV 檔案
    output_filename = f"week_{w}_output.csv"
    df.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"資料已成功匯出至 {output_filename}")
